In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

import tensorflow as tf
import pandas as pd
import numpy as np

# sys.path.append('./GRU-D/')
# from models import create_grud_model
# import seaborn as sns
import matplotlib.pyplot as plt
import json
PROJECT_SRC_DIR = '/cluster/tufts/hugheslab/prath01/projects/time_series_prediction/src/'
sys.path.append(PROJECT_SRC_DIR)
sys.path.append(os.path.join(PROJECT_SRC_DIR, "rnn"))
sys.path.append(os.path.abspath("../src"))
from dataset_loader import TidySequentialDataCSVLoader
from feature_transformation import parse_id_cols, parse_output_cols, parse_feature_cols, parse_id_cols, parse_time_cols
%matplotlib inline

## Load MIMIC data

In [2]:
train_test_dir = '/cluster/tufts/hugheslab/prath01/projects/time_series_prediction/datasets/mimic3_inhospital_mortality/v20201207/split-by=subject_id/features_per_timeslice/classifier_train_test_split_dir/'
x_dict_file = '/cluster/tufts/hugheslab/prath01/projects/time_series_prediction/datasets/mimic3_inhospital_mortality/v20201207/split-by=subject_id/features_per_timeslice/classifier_train_test_split_dir/x_dict.json'
y_dict_file = '/cluster/tufts/hugheslab/prath01/projects/time_series_prediction/datasets/mimic3_inhospital_mortality/v20201207/split-by=subject_id/features_per_timeslice/classifier_train_test_split_dir/y_dict.json'

x_train_csv_filename = os.path.join(train_test_dir, 'x_train_first_24_hours.csv')
y_train_csv_filename = os.path.join(train_test_dir, 'y_train_first_24_hours.csv')
# x_train_df = pd.read_csv(x_train_csv_filename)
# y_train_df = pd.read_csv(y_train_csv_filename)

x_valid_csv_filename = os.path.join(train_test_dir, 'x_valid_first_24_hours.csv')
y_valid_csv_filename = os.path.join(train_test_dir, 'y_valid_first_24_hours.csv')
# x_valid_df = pd.read_csv(x_valid_csv_filename)
# y_valid_df = pd.read_csv(y_valid_csv_filename)

x_test_csv_filename = os.path.join(train_test_dir, 'x_test_first_24_hours.csv')
y_test_csv_filename = os.path.join(train_test_dir, 'y_test_first_24_hours.csv')
# x_test_df = pd.read_csv(x_test_csv_filename)
# y_test_df = pd.read_csv(y_test_csv_filename)


train_x = pd.read_csv(x_train_csv_filename)
valid_x = pd.read_csv(x_valid_csv_filename)
test_x = pd.read_csv(x_test_csv_filename)
train_y = pd.read_csv(y_train_csv_filename)
valid_y = pd.read_csv(y_valid_csv_filename)
test_y = pd.read_csv(y_test_csv_filename)

x_data_dict = json.load(open(x_dict_file))
y_data_dict = json.load(open(y_dict_file))

feature_cols = parse_feature_cols(x_data_dict['schema'])
id_cols = parse_id_cols(x_data_dict['schema'])

In [6]:
train_x.iloc[:26, :20]

,subject_id,hadm_id,icustay_id,hours_in,age,alanine aminotransferase,albumin,albumin ascites,albumin pleural,albumin urine,alkaline phosphate,anion gap,asparate aminotransferase,basophils,bicarbonate,bilirubin,blood urea nitrogen,calcium,calcium ionized,calcium urine
0,3,145834,211552,0,76.526794,25.0,1.8,NaN,NaN,NaN,73.0,20.666666,69.0,NaN,16.333334,0.8,44.2,6.92,0.963333,NaN
1,3,145834,211552,1,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.060000,NaN
3,3,145834,211552,3,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,4,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,145834,211552,5,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3,145834,211552,6,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,1.030000,NaN
7,3,145834,211552,7,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,NaN,13.000000,NaN,42.0,6.70,NaN,NaN
8,3,145834,211552,8,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,145834,211552,9,76.526794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train_vitals = TidySequentialDataCSVLoader(
    x_csv_path=train_x,
    y_csv_path=train_y,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort_hosp",
    y_label_type='per_sequence'
)

valid_vitals = TidySequentialDataCSVLoader(
    x_csv_path=valid_x,
    y_csv_path=valid_y,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort_hosp",
    y_label_type='per_sequence'
)

test_vitals = TidySequentialDataCSVLoader(
    x_csv_path=test_x,
    y_csv_path=test_y,
    x_col_names=feature_cols,
    idx_col_names=id_cols,
    y_col_name="mort_hosp",
    y_label_type='per_sequence'
)

# num_true_feats = int(F/3)
train_x_NTD, y_train = train_vitals.get_batch_data(batch_id=0)
valid_x_NTD, y_valid = valid_vitals.get_batch_data(batch_id=0)
test_x_NTD, y_test = test_vitals.get_batch_data(batch_id=0)

N_tr = len(train_x_NTD)
N_va = len(valid_x_NTD)
N_te = len(test_x_NTD)

In [15]:
for split, y in [('train', y_train),
                ('valid', y_valid),
                ('test', y_test)]:
    print('fraction positive labels in %s set : %.4f'%(split, y.sum()/len(y)))



fraction positive labels in train set : 0.0965
fraction positive labels in valid set : 0.0938
fraction positive labels in test set : 0.0993


In [11]:
state_id = 41
data_save_dir = '/cluster/tufts/hugheslab/prath01/datasets/mimic3_ssl/'

for ii, perc_labelled in enumerate([1.2]):#3.7, 11.1, 33.3, 100
    curr_save_dir = os.path.join(data_save_dir, 'percentage_labelled_sequnces=%s'%perc_labelled)
    
    print('---------------------------------------------------------------------------')
    print('CREATING TRAIN/VALID/TEST SPLITS FOR %.3f PERCENT OF SEQUENCES LABELLED'%perc_labelled)
    print('---------------------------------------------------------------------------')
    y_train_ss = y_train.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_tr = int((1-(perc_labelled)/100)*N_tr)
    unlabelled_inds_tr = rnd_state.permutation(N_tr)[:n_unlabelled_tr]
    y_train_ss = y_train_ss.astype(np.float32)
    y_train_ss[unlabelled_inds_tr] = np.nan  
    if perc_labelled!=100:
        print('Excluded inds train: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_tr[0],
                                                              unlabelled_inds_tr[1],
                                                              unlabelled_inds_tr[2],
                                                              unlabelled_inds_tr[-3],
                                                              unlabelled_inds_tr[-2],
                                                              unlabelled_inds_tr[-1]))
    
    y_valid_ss = y_valid.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_va = int((1-(perc_labelled)/100)*N_va)
    unlabelled_inds_va = rnd_state.permutation(N_va)[:n_unlabelled_va]
    y_valid_ss = y_valid_ss.astype(np.float32)
    y_valid_ss[unlabelled_inds_va] = np.nan 
    if perc_labelled!=100:
        print('Excluded inds valid: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_va[0],
                                                          unlabelled_inds_va[1],
                                                          unlabelled_inds_va[2],
                                                          unlabelled_inds_va[-3],
                                                          unlabelled_inds_va[-2],
                                                          unlabelled_inds_va[-1]))

    y_test_ss = y_test.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_te = int((1-(perc_labelled)/100)*N_te)
    unlabelled_inds_te = rnd_state.permutation(N_te)[:n_unlabelled_te]
    y_test_ss = y_test_ss.astype(np.float32)
    y_test_ss[unlabelled_inds_te] = np.nan
    if perc_labelled!=100:
        print('Excluded inds test: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_te[0],
                                                          unlabelled_inds_te[1],
                                                          unlabelled_inds_te[2],
                                                          unlabelled_inds_te[-3],
                                                          unlabelled_inds_te[-2],
                                                          unlabelled_inds_te[-1]))
    
    # Check whether the specified path exists or not
    isExist = os.path.exists(curr_save_dir)

    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(curr_save_dir)
        
    # save the data to the respective folder
    print('Saving data to %s'%curr_save_dir)
    np.save(os.path.join(curr_save_dir, 'X_train.npy'), train_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_train.npy'), y_train_ss)
    print('Done saving train..')
    np.save(os.path.join(curr_save_dir, 'X_valid.npy'), valid_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_valid.npy'), y_valid_ss)
    print('Done saving valid..')
    np.save(os.path.join(curr_save_dir, 'X_test.npy'), test_x_NTD)
    np.save(os.path.join(curr_save_dir, 'y_test.npy'), y_test_ss)
    print('Done saving test..')
    
    
    print('---------------------------------------------------------------------------')
    for split, y in [('train', y_train_ss),
                    ('valid', y_valid_ss),
                    ('test', y_test_ss)]:
        frac_pos_labels = np.nansum(y)/(~np.isnan(y)).sum()
        print('fraction positive labels in %s set with %.3f percent of sequences labelled : %.4f'%(split,
                                                                                                   perc_labelled,
                                                                                                   frac_pos_labels))
    print('---------------------------------------------------------------------------')

---------------------------------------------------------------------------
CREATING TRAIN/VALID/TEST SPLITS FOR 1.200 PERCENT OF SEQUENCES LABELLED
---------------------------------------------------------------------------
Excluded inds train: 2995, 957, 4074 ... 11398, 9140, 18547
Excluded inds valid: 671, 846, 2062 ... 1448, 3199, 174
Excluded inds test: 1383, 2603, 5628 ... 3199, 174, 434
Saving data to /cluster/tufts/hugheslab/prath01/datasets/mimic3_ssl/percentage_labelled_sequnces=1.2
Done saving train..
Done saving valid..
Done saving test..
---------------------------------------------------------------------------
fraction positive labels in train set with 1.200 percent of sequences labelled : 0.0933
fraction positive labels in valid set with 1.200 percent of sequences labelled : 0.0351
fraction positive labels in test set with 1.200 percent of sequences labelled : 0.0423
---------------------------------------------------------------------------


In [52]:
x_valid_NTD_loaded = np.load('X_valid.npy')

In [56]:
x_valid_NTD_loaded[:3]

array([[[ 50.84156,  45.     ,   4.9    , ...,       nan,  10.4    ,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        [300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        [300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        [300.00308,       nan,       nan, ...,       nan,       nan,
   

In [57]:
valid_x_NTD[:3]

array([[[ 50.84156,  45.     ,   4.9    , ...,       nan,  10.4    ,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan],
        [ 50.84156,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        [300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        [300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [300.00308,       nan,       nan, ...,       nan,       nan,
               nan],
        [300.00308,       nan,       nan, ...,       nan,       nan,
   

## Load the MIMIC collapsed features data

In [4]:
from featurize_single_time_series import collapse_std, collapse_elapsed_time_since_last_measured, collapse_count, collapse_slope, collapse_median, collapse_min, collapse_max, collapse_value_last_measured, make_summary_ops

In [5]:
# collapse features
def featurize_ts(
        time_arr_by_var,
        val_arr_by_var,
        n_features,
        percentile_slices_to_featurize=[(0., 100.)],
        summary_ops=['count', 'mean', 'std', 'slope'],
        ):
    ''' Featurize provided multivariate irregular time series into flat vector
    Args
    ----
    time_arr_by_var : dict of 1D NumPy arrays
    val_arr_by_var : dict of 1D NumPy arrays
    start_numerictime : float
        Indicates numerical time value at which current window *starts*
    stop_numerictime : float
        Indicates numerical time that current window *stops*
    Returns
    -------
    feat_vec_1F : 2D NumPy array, shape (1, F)
        One entry for each combination of {variable, summary op, subwindow slice}
    '''
    
    start_numerictime = 0
    stop_numerictime = 24
    time_range = stop_numerictime - start_numerictime

    F = len(percentile_slices_to_featurize) * n_features * len (summary_ops)
    feat_vec_1F = np.zeros((1, F))
    ff = 0

    SUMMARY_OPERATIONS = make_summary_ops()

    for rp_ind, (low, high) in enumerate(percentile_slices_to_featurize):
        cur_window_start_time = start_numerictime + float(low) / 100 * time_range
        cur_window_stop_time = start_numerictime + float(high) / 100 * time_range

        for var_id in range(n_features):
            cur_feat_arr = val_arr_by_var[:, var_id].astype('float')
            cur_numerictime_arr = time_arr_by_var

            # Keep only the entries whose times occur within current window
            start = np.searchsorted(
                cur_numerictime_arr, cur_window_start_time, side='left')
            stop = np.searchsorted(
                cur_numerictime_arr, cur_window_stop_time, side='right')
            cur_numerictime_arr = cur_numerictime_arr[start:stop]
            cur_feat_arr = cur_feat_arr[start:stop]
            cur_isfinite_arr = np.isfinite(cur_feat_arr)
            
            for op_ind, op in enumerate(summary_ops):
                summary_func, empty_val = SUMMARY_OPERATIONS[op]
                if cur_feat_arr.size < 1 or cur_isfinite_arr.sum() < 1:
                    feat_vec_1F[0,ff] = empty_val
                else:
                    feat_vec_1F[0,ff] = summary_func(
                        cur_feat_arr, cur_numerictime_arr, cur_isfinite_arr,
                        cur_window_start_time, cur_window_stop_time)
#                 feat_names.append("feature_%s_%s_%.0f-%.0f" % (var_id, op, float(low), float(high)))
                ff += 1
    return feat_vec_1F

In [6]:
N_tr = len(train_x_NTD)
N_va = len(valid_x_NTD)
N_te = len(test_x_NTD)
percentile_slices_to_featurize = [(0., 100.), (90, 100)]
summary_ops = ["std", "time_since_measured", "count", "slope", "median", "min", "max", "last_value_measured"]
n_features = train_x_NTD.shape[-1]
F = len(percentile_slices_to_featurize) * n_features * len (summary_ops)


train_x_collapsed_NF = np.zeros((N_tr, F))
valid_x_collapsed_NF = np.zeros((N_va, F))
test_x_collapsed_NF = np.zeros((N_te, F))

print('Collapsing train feaures')
for nn in range(N_tr):
    if (nn%500)==0:
        print('Done with %s sequences..'%nn)
    train_x_collapsed_NF[nn, :] = featurize_ts(np.arange(0, 25).astype(float),
                                               train_x_NTD[nn],
                                               n_features,
                                               percentile_slices_to_featurize=[(0., 100.), (90, 100)],
                                               summary_ops=["std", "time_since_measured", 
                                                            "count", "slope", "median", 
                                                            "min", "max", "last_value_measured"])
print('Collapsing valid feaures')
for nn in range(N_va):
    if (nn%500)==0:
        print('Done with %s sequences..'%nn)
    valid_x_collapsed_NF[nn, :] = featurize_ts(np.arange(0, 25).astype(float),
                                               valid_x_NTD[nn],
                                               n_features,
                                               percentile_slices_to_featurize=[(0., 100.), (90, 100)],
                                               summary_ops=["std", "time_since_measured", 
                                                            "count", "slope", "median", 
                                                            "min", "max", "last_value_measured"])

print('Collapsing test feaures')
for nn in range(N_te):
    if (nn%500)==0:
        print('Done with %s sequences..'%nn)
    test_x_collapsed_NF[nn, :] = featurize_ts(np.arange(0, 25).astype(float),
                                               test_x_NTD[nn],
                                               n_features,
                                               percentile_slices_to_featurize=[(0., 100.), (90, 100)],
                                               summary_ops=["std", "time_since_measured", 
                                                            "count", "slope", "median", 
                                                            "min", "max", "last_value_measured"])

Collapsing train feaures
Done with 0 sequences..
Done with 500 sequences..
Done with 1000 sequences..
Done with 1500 sequences..
Done with 2000 sequences..
Done with 2500 sequences..
Done with 3000 sequences..
Done with 3500 sequences..
Done with 4000 sequences..
Done with 4500 sequences..
Done with 5000 sequences..
Done with 5500 sequences..
Done with 6000 sequences..
Done with 6500 sequences..
Done with 7000 sequences..
Done with 7500 sequences..
Done with 8000 sequences..
Done with 8500 sequences..
Done with 9000 sequences..
Done with 9500 sequences..
Done with 10000 sequences..
Done with 10500 sequences..
Done with 11000 sequences..
Done with 11500 sequences..
Done with 12000 sequences..
Done with 12500 sequences..
Done with 13000 sequences..
Done with 13500 sequences..
Done with 14000 sequences..
Done with 14500 sequences..
Done with 15000 sequences..
Done with 15500 sequences..
Done with 16000 sequences..
Done with 16500 sequences..
Done with 17000 sequences..
Done with 17500 seq

In [7]:
state_id = 41
data_save_dir = '/cluster/tufts/hugheslab/prath01/datasets/mimic3_ssl/'

for ii, perc_labelled in enumerate([1.2]):#3.7, 11.1, 33.3, 100
    curr_save_dir = os.path.join(data_save_dir, 'percentage_labelled_sequnces=%s'%perc_labelled)
    
    print('---------------------------------------------------------------------------')
    print('CREATING TRAIN/VALID/TEST SPLITS FOR %.3f PERCENT OF SEQUENCES LABELLED'%perc_labelled)
    print('---------------------------------------------------------------------------')
    y_train_ss = y_train.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_tr = int((1-(perc_labelled)/100)*N_tr)
    unlabelled_inds_tr = rnd_state.permutation(N_tr)[:n_unlabelled_tr]
    y_train_ss = y_train_ss.astype(np.float32)
    y_train_ss[unlabelled_inds_tr] = np.nan  
    if perc_labelled!=100:
        print('Excluded inds train: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_tr[0],
                                                              unlabelled_inds_tr[1],
                                                              unlabelled_inds_tr[2],
                                                              unlabelled_inds_tr[-3],
                                                              unlabelled_inds_tr[-2],
                                                              unlabelled_inds_tr[-1]))
    
    y_valid_ss = y_valid.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_va = int((1-(perc_labelled)/100)*N_va)
    unlabelled_inds_va = rnd_state.permutation(N_va)[:n_unlabelled_va]
    y_valid_ss = y_valid_ss.astype(np.float32)
    y_valid_ss[unlabelled_inds_va] = np.nan 
    if perc_labelled!=100:
        print('Excluded inds valid: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_va[0],
                                                          unlabelled_inds_va[1],
                                                          unlabelled_inds_va[2],
                                                          unlabelled_inds_va[-3],
                                                          unlabelled_inds_va[-2],
                                                          unlabelled_inds_va[-1]))

    y_test_ss = y_test.copy()
    rnd_state = np.random.RandomState(state_id)
    n_unlabelled_te = int((1-(perc_labelled)/100)*N_te)
    unlabelled_inds_te = rnd_state.permutation(N_te)[:n_unlabelled_te]
    y_test_ss = y_test_ss.astype(np.float32)
    y_test_ss[unlabelled_inds_te] = np.nan
    if perc_labelled!=100:
        print('Excluded inds test: %d, %d, %d ... %d, %d, %d'%(unlabelled_inds_te[0],
                                                          unlabelled_inds_te[1],
                                                          unlabelled_inds_te[2],
                                                          unlabelled_inds_te[-3],
                                                          unlabelled_inds_te[-2],
                                                          unlabelled_inds_te[-1]))
    
    # Check whether the specified path exists or not
    isExist = os.path.exists(curr_save_dir)

    if not isExist:
        # Create a new directory because it does not exist 
        os.makedirs(curr_save_dir)
        
    # save the data to the respective folder
    print('Saving data to %s'%curr_save_dir)
    np.save(os.path.join(curr_save_dir, 'X_train_collapsed.npy'), train_x_collapsed_NF)
    np.save(os.path.join(curr_save_dir, 'y_train_collapsed.npy'), y_train_ss)
    print('Done saving train..')
    np.save(os.path.join(curr_save_dir, 'X_valid_collapsed.npy'), valid_x_collapsed_NF)
    np.save(os.path.join(curr_save_dir, 'y_valid_collapsed.npy'), y_valid_ss)
    print('Done saving valid..')
    np.save(os.path.join(curr_save_dir, 'X_test_collapsed.npy'), test_x_collapsed_NF)
    np.save(os.path.join(curr_save_dir, 'y_test_collapsed.npy'), y_test_ss)
    print('Done saving test..')


---------------------------------------------------------------------------
CREATING TRAIN/VALID/TEST SPLITS FOR 1.200 PERCENT OF SEQUENCES LABELLED
---------------------------------------------------------------------------
Excluded inds train: 2995, 957, 4074 ... 11398, 9140, 18547
Excluded inds valid: 671, 846, 2062 ... 1448, 3199, 174
Excluded inds test: 1383, 2603, 5628 ... 3199, 174, 434
Saving data to /cluster/tufts/hugheslab/prath01/datasets/mimic3_ssl/percentage_labelled_sequnces=1.2
Done saving train..
Done saving valid..
Done saving test..


In [6]:
3.7/3

1.2333333333333334